# **Pyspark**

#### Importa bibliotecas

In [5]:
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark import SparkConf
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql.functions import countDistinct
from pyspark.sql import Window
import os

#### Cria schema

In [6]:

schema = StructType(
   [StructField('id', StringType(), True),
    StructField('data', StringType(), True),
    StructField('dia_semana', StringType(), True),
    StructField('uf', StringType(), True),
    StructField('rodovia', StringType(), True),
    StructField('km', FloatType(), True),
    StructField('municipio', StringType(), True),
    StructField('causa_acidente', StringType(), True),
    StructField('tipo_acidente', StringType(), True),
    StructField('classificacao_acidente', StringType(), True),
    StructField('fase_dia', StringType(), True),
    StructField('sentido_via', StringType(), True),
    StructField('condicao_metereologica', StringType(), True),
    StructField('tipo_pista', StringType(), True),
    StructField('tracado_via', StringType(), True),
    StructField('uso_solo', StringType(), True),
    StructField('pessoas', IntegerType(), True),
    StructField('mortos', IntegerType(), True),
    StructField('feridos_leves', IntegerType(), True),
    StructField('feridos_graves', IntegerType(), True),
    StructField('ilesos', IntegerType(), True),
    StructField('ignorados', IntegerType(), True),
    StructField('feridos', IntegerType(), True),
    StructField('veiculos', IntegerType(), True),
    StructField('latitude', StringType(), True),
    StructField('longitude', StringType(), True),
  
   ]
  )

In [ ]:
df = spark.read.options(header=True).option("delimiter", ";").option("encoding", "ISO-8859-1").csv("Projeto Final/acidentes_ocorrencias.csv", schema=schema)
df.show()

#### Verifica únicos para condição metereológica

In [40]:
df.groupBy(df.condicao_metereologica).count().orderBy('count', ascending=False).show(150)

+----------------------+------+
|condicao_metereologica| count|
+----------------------+------+
|             Céu Claro|202539|
|               Nublado| 61714|
|                 Chuva| 42675|
|                   Sol| 26481|
|        Garoa/Chuvisco| 12016|
|              Ignorado|  5166|
|      Nevoeiro/Neblina|  2915|
|                 Vento|   770|
|               Granizo|    11|
|                  Neve|     6|
+----------------------+------+



In [41]:
df.select('condicao_metereologica').distinct().collect()

[Row(condicao_metereologica='Vento'),
 Row(condicao_metereologica='Ignorado'),
 Row(condicao_metereologica='Garoa/Chuvisco'),
 Row(condicao_metereologica='Nevoeiro/Neblina'),
 Row(condicao_metereologica='Chuva'),
 Row(condicao_metereologica='Neve'),
 Row(condicao_metereologica='Nublado'),
 Row(condicao_metereologica='Sol'),
 Row(condicao_metereologica='Céu Claro'),
 Row(condicao_metereologica='Granizo')]

####  Verifica valores para traçado da via

In [42]:
df.select('tracado_via').distinct().collect()

[Row(tracado_via='Reta'),
 Row(tracado_via='Viaduto'),
 Row(tracado_via='Não Informado'),
 Row(tracado_via='Interseção de vias'),
 Row(tracado_via='Curva'),
 Row(tracado_via='Rotatória'),
 Row(tracado_via='Túnel'),
 Row(tracado_via='Retorno Regulamentado'),
 Row(tracado_via='Ponte'),
 Row(tracado_via='Desvio Temporário')]

In [43]:
df.groupBy(df.tracado_via).count().orderBy('count', ascending=False).show(150)

+--------------------+------+
|         tracado_via| count|
+--------------------+------+
|                Reta|215175|
|               Curva| 59196|
|       Não Informado| 38046|
|  Interseção de vias| 15912|
|   Desvio Temporário|  9848|
|           Rotatória|  7127|
|Retorno Regulamen...|  4102|
|             Viaduto|  2510|
|               Ponte|  1963|
|               Túnel|   414|
+--------------------+------+



#### Cria nova coluna que agrupa as causas de acidentes
- Falha Humana
- Falha Mecânica
- Fenômenos da natureza
- Infraestrutura da pista
- Pedestre
- Outros

In [ ]:
df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show(vertical=True)

In [ ]:
b=[x[0] for x in df.select('causa_acidente').distinct().collect()]

In [ ]:
condicao=when( (col('causa_acidente')=='Falta de Atenção à Condução')|
               (col('causa_acidente')=='Velocidade Incompatível')|
               (col('causa_acidente')=='Mal Súbito') |
               (col('causa_acidente')=='Não guardar distância de segurança')|
               (col('causa_acidente')=='Ultrapassagem Indevida')|
               (col('causa_acidente')=='Ingestão de Substâncias Psicoativas')|
               (col('causa_acidente')=='Desobediência às normas de trânsito pelo condutor')|
               (col('causa_acidente')=='Carga excessiva e/ou mal acondicionada')|
               (col('causa_acidente')=='Transitar no acostamento')|
               (col('causa_acidente')=='Conversão proibida')|
               (col('causa_acidente')=='Estacionar ou parar em local proibido')|
               (col('causa_acidente')== 'Transitar no acostamento')|
               (col('causa_acidente')=='Reação tardia ou ineficiente do condutor')|
               (col('causa_acidente')=='Ausência de reação do condutor')|
               (col('causa_acidente')=='Desrespeitar a preferência no cruzamento')|
               (col('causa_acidente')=='Condutor deixou de manter distância do veículo da frente')|
               (col('causa_acidente')=='Acessar a via sem observar a presença dos outros veículos')|
               (col('causa_acidente')=='Mal súbito do condutor')|
               (col('causa_acidente')=='Transitar na contramão')|
               (col('causa_acidente')== 'Condutor desrespeitou a iluminação vermelha do semáforo')|
               (col('causa_acidente')=='Condutor Dormindo')|
               (col('causa_acidente')=='Deficiência ou não Acionamento do Sistema de Iluminação/Sinalização do Veículo')|
               (col('causa_acidente')=='Ingestão de Álcool')|
               (col('causa_acidente')=='Ingestão de álcool pelo condutor')|
               (col('causa_acidente')=='Manobra de mudança de faixa')|
               (col('causa_acidente')=='Ingestão de substâncias psicoativas pelo condutor')|
               (col('causa_acidente')=='Trafegar com motocicleta (ou similar) entre as faixas')|
               (col('causa_acidente')=='Deixar de acionar o farol da motocicleta (ou similar)')|
               (col('causa_acidente')=='Participar de racha')|
               (col('causa_acidente')=='Frear bruscamente')|
               (col('causa_acidente')=='Retorno proibido')|
               (col('causa_acidente')=='Transitar na calçada')|
               (col('causa_acidente')=='Condutor usando celular'),'Falha Humana').when((col('causa_acidente')=='Demais falhas mecânicas ou elétricas')|
               (col('causa_acidente')=='Demais falhas mecânicas ou elétricas')|
               (col('causa_acidente')=='Modificação proibida')|
               (col('causa_acidente')=='Avarias e/ou desgaste excessivo no pneu')|
               (col('causa_acidente')=='Problema na suspensão')|
               (col('causa_acidente')=='Problema com o freio')|
               (col('causa_acidente')=='Defeito Mecânico no Veículo')|
               (col('causa_acidente')== 'Faróis desregulados'), 'Falha Mecânica').when((col('causa_acidente')== 'Chuva')|
               (col('causa_acidente')=='Restrição de Visibilidade')|
               (col('causa_acidente')=='Animais na Pista')|
               (col('causa_acidente')=='Neblina')|
               (col('causa_acidente')=='Fenômenos da Natureza')|
               (col('causa_acidente')=='Demais Fenômenos da natureza')|
               (col('causa_acidente')=='Fumaça'), 'Fenômenos da Natureza').when((col('causa_acidente')=='Curva acentuada')|
               (col('causa_acidente')== 'Ausência de sinalização')|(col('causa_acidente')=='Curva acentuada')|
               (col('causa_acidente')=='Acesso irregular')|
               (col('causa_acidente')=='Desvio temporário')|
               (col('causa_acidente')=='Acostamento em desnível')|
               (col('causa_acidente')=='Área urbana sem a presença de local apropriado para a travessia de pedestres')|
               (col('causa_acidente')=='Pista esburacada')|
               (col('causa_acidente')=='Semáforo com defeito')|(col('causa_acidente')=='Sinalização encoberta')|
               (col('causa_acidente')=='Pista Escorregadia')|(col('causa_acidente')=='Defeito na Via')|
               (col('causa_acidente')=='Sinalização da via insuficiente ou inadequada')|(col('causa_acidente')=='Modificação proibida')|
               (col('causa_acidente')=='Acumulo de areia ou detritos sobre o pavimento')|
               (col('causa_acidente')=='Iluminação deficiente')|(col('causa_acidente')=='Afundamento ou ondulação no pavimento')|
               (col('causa_acidente')=='Acumulo de água sobre o pavimento')|(col('causa_acidente')=='Sistema de drenagem ineficiente')|
               (col('causa_acidente')=='Redutor de velocidade em desacordo')|
               (col('causa_acidente')=='Acumulo de óleo sobre o pavimento')|(col('causa_acidente')=='Declive acentuado')|(col('causa_acidente')=='Faixas de trânsito com largura insuficiente')|
               (col('causa_acidente')=='Sistema de drenagem ineficiente')|
               (col('causa_acidente')=='Deficiência do Sistema de Iluminação/Sinalização')|
               (col('causa_acidente')=='Restrição de visibilidade em curvas verticais')|
               (col('causa_acidente')=='Falta de acostamento')| (col('causa_acidente')=='Sinalização mal posicionada')|(col('causa_acidente')=='Pista em desnível')|(col('causa_acidente')== 'Obras na pista')|
               (col('causa_acidente')=='Demais falhas na via'), 'Infraestrutura da Pista') .when((col('causa_acidente')=='Ingestão de álcool e/ou substâncias psicoativas pelo pedestre')|
               (col('causa_acidente')=='Pedestre cruzava a pista fora da faixa')|
               (col('causa_acidente')=='Entrada inopinada do pedestre')|(col('causa_acidente')=='Ingestão de álcool ou de substâncias psicoativas pelo pedestre')|
               (col('causa_acidente')=='Falta de Atenção do Pedestre')|
               (col('causa_acidente')=='Pedestre andava na pista')|
               (col('causa_acidente')=='Desobediência às normas de trânsito pelo pedestre'), 'Pedestre').otherwise('Outros')

In [ ]:
df=df.withColumn("causa_grupos", condicao)

#### Cria coluna que diz se a caysa do acidente está relacionada ou não ao consumo de álcool

In [ ]:
df=df.withColumn("consumo_alcool", F.col('causa_acidente').rlike("Álcool"))
df=df.withColumn('consumo_alcool', when(df.consumo_alcool=='true', 'sim').otherwise('não'))